### Changing the working directory

In [1]:
import os

In [2]:
%pwd

'/home/adityasanyal1996/Movie_Recommendation/research'

In [3]:
os.chdir("../")

In [7]:
%pwd

'/home/adityasanyal1996/Movie_Recommendation'

In [9]:
import pandas as pd

df = pd.read_csv('dataset/netflix_dataset.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


### Config entity

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataValidationConfig:
    root_dir: Path
    status_file: str
    local_data_file: Path
    all_schema: dict

### Configuration Manager

In [6]:
from recommender.constants import *
from recommender.utils.common import create_directories, read_yaml

In [10]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])
        data_validation_config = DataValidationConfig(
            root_dir = config.root_dir,
            status_file = config.status_file,
            local_data_file = config.local_data_file,
            all_schema = schema
        )
        return data_validation_config

### Component

In [12]:
import os
from recommender import logger

In [16]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_columns(self)->bool:
        try:
            validation_status = True
            with open(self.config.status_file, 'w') as f:
                pass

            data = pd.read_csv(self.config.local_data_file)
            all_cols = list(data.columns)
            all_schema_keys = self.config.all_schema.keys()

            #for validating the data types
            size = len(all_schema_keys)
            count = 0
            for col in all_cols:
                count += 1
                for key in list(all_schema_keys):
                    if(col == key):
                        if(str(data.dtypes[col]) != (self.config.all_schema)[key]):
                            validation_status = False
                            logger.info(f"{col} type in dataset doesn't match with {key} in schema")
                            break
                if count == size:
                    pass
                else:
                    continue

            with open(self.config.status_file, 'a') as f:
                f.write(f"Type Validation status: {validation_status}")

                f.write("\n")

            #for validating the columns
            validation_status = True
            for col in all_cols:
                if col not in all_schema_keys:
                    validation_status = False
                    logger.info(f"{col} column doesn't match with the schema")
            
            with open(self.config.status_file, 'a') as f:
                f.write(f"Column Validation status: {validation_status}")

            return validation_status
        except Exception as e:
            raise e

### Pipeline

In [21]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2024-08-04 18:43:17,068: INFO: common: yaml file:config/config.yaml loaded successfully]
[2024-08-04 18:43:17,072: INFO: common: yaml file:schema.yaml loaded successfully]
[2024-08-04 18:43:17,073: INFO: common: create directory at artifacts]
[2024-08-04 18:43:17,074: INFO: common: create directory at artifacts/data_validation]
